In [1]:
import pandas as pd
import json
from pprint import pprint
from sqlalchemy import create_engine

In [2]:
#read in csv utilizing pandas
poverty_csv=pd.read_csv('countypoverty.csv')
#drop all irrelevant columns
fixed_csv=poverty_csv.drop(['FIPStxt','CI90LBAll_2018','CI90UBALL_2018','CI90LBALLP_2018','CI90UBALLP_2018','CI90LB017_2018','CI90UB017_2018','CI90LB017P_2018','CI90UB017P_2018','POV517_2018','CI90LB517_2018',
               'CI90UB517_2018','PCTPOV517_2018','CI90LB517P_2018','CI90UB517P_2018','CI90LBINC_2018','CI90UBINC_2018','POV04_2018','CI90LB04_2018','CI90UB04_2018','PCTPOV04_2018','CI90LB04P_2018',
               'CI90UB04P_2018'],axis=1)
#rename relevant columns appropriately
poverty_df=fixed_csv.rename(columns={'Stabr':'abbreviation','Area_name':'area_name','POVALL_2018':'impoverished_population','PCTPOVALL_2018':'percentage_of_impoverished_population','POV017_2018':'impoverished_children','PCTPOV017_2018':'percentage_of_imporverished_children','MEDHHINC_2018':'median_household_income'})
poverty_df=poverty_df.rename_axis(index='id')
poverty_df=poverty_df.reset_index()
poverty_df.set_index("id",inplace=True)

In [3]:
#yelp_json=pd.read_json('yelp_academic_dataset_business.json',lines=True)
#fixed_json=yelp_json.drop(['business_id','address','postal_code','latitude','longitude','attributes','hours'],axis=1)
#yelp_df=fixed_json.rename(columns={'name':'Business Name','city':'City','state':'State','stars':'Rating out of 5','review_count':'Review Count','is_open':'Open?','categories':'Categories'})
county=[]
state=[]
enrollees=[]
with open('VAEnrolleesByCountyFY2015.json') as json_file:
    data=json.load(json_file)
    for d in data['DataByCounty']:
        county.append(d['CountyName'])
        state.append(d['StateAbbrev'])
        enrollees.append(d['VeteranEnrollees'])
almost=pd.DataFrame(list(zip(county, state,enrollees)),)
prettymuch=almost.rename(columns={0:'county',1:'state',2:'va_enrollees'})
#prettymuch.loc[prettymuch['State']!=['GU','PR','VI']]
once=prettymuch.loc[prettymuch['state']!='GU']
twice=once.loc[once['state']!='PR']
threetimes=twice.loc[twice['state']!='VI']
va_df=threetimes.loc[threetimes['county']!='Unassigned']
va_df=va_df.rename_axis(index='id')
va_df=va_df.reset_index()
va_df.set_index("id", inplace=True)

In [5]:
connection_string = "postgres:postgres@localhost:5432/ETL_db"
engine = create_engine(f'postgresql://{connection_string}')
engine.table_names()

['poverty', 'enrollees']

In [7]:
poverty_df.to_sql(name='poverty', con=engine, if_exists='append', index=True)
va_df.to_sql(name='enrollees', con=engine, if_exists='append', index=True)